In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install --upgrade seeme

  Attempting uninstall: seeme
    Found existing installation: seeme 0.16.2
    Uninstalling seeme-0.16.2:
      Successfully uninstalled seeme-0.16.2


In [3]:
from seeme import Client

cl = Client(backend="http://192.168.0.3:5001/api/v1")

username = ""
password = ""
cl.login(username, password)

In [4]:
# Dataset

In [5]:
ds = cl.get_datasets()

In [6]:
[d for d in ds if "Adult" in d["name"]]

[{'id': '224315a3-3daf-4f38-bd5c-53dd2a8c8f71',
  'created_at': '2022-09-27T10:51:02.381408+02:00',
  'updated_at': '2022-09-27T10:51:08.333243+02:00',
  'name': 'Adult dataset',
  'description': 'Wage classification',
  'user_id': '2b02eb44-8ad0-409c-9515-0405fb25d470',
  'notes': '',
  'has_logo': False,
  'logo': '',
  'public': False,
  'shared_with_me': False,
  'versions': None,
  'multi_label': False,
  'default_splits': False,
  'content_type': 'tabular'},
 {'id': 'b753359a-25da-4ca9-bfac-94b3da258d79',
  'created_at': '2022-09-27T09:26:40.585755+02:00',
  'updated_at': '2022-09-27T09:26:44.565163+02:00',
  'name': 'Adult dataset',
  'description': 'Wage classification',
  'user_id': '2b02eb44-8ad0-409c-9515-0405fb25d470',
  'notes': '',
  'has_logo': False,
  'logo': '',
  'public': False,
  'shared_with_me': False,
  'versions': None,
  'multi_label': False,
  'default_splits': False,
  'content_type': 'tabular'},
 {'id': '9573ea77-e146-42b6-a5ec-092e7d5b893c',
  'created_at'

In [7]:
adult_dataset = {
    "name": "Adult dataset",
    "description": "Wage classification"
}


adult_dataset = cl.create_dataset(adult_dataset)

In [8]:
ds_version = adult_dataset["versions"][0]

In [9]:
d_id = adult_dataset["id"]
v_id = ds_version["id"]

In [10]:
csv_file = "tabular_adult_csv.zip"
#csv_file = "tabular_simple.csv.zip"

In [11]:
from seeme import DATASET_FORMAT_CSV

In [12]:
ds_version = cl.upload_dataset_version(d_id, v_id, folder=".", filename=csv_file, format=DATASET_FORMAT_CSV )

In [13]:
# Train Job

In [14]:
from seeme import JOB_STATUS_WAITING

In [15]:
import torch
import fastai
#-- Application ID --
application_id = cl.get_application_id(
    base_framework="pytorch",
    framework="fastai",
    base_framework_version=str(torch.__version__),
    framework_version=str(fastai.__version__),
    application="structured"
)

In [16]:
str(torch.__version__)

'1.12.1'

In [49]:
from seeme import JOB_TYPE_TRAINING

job_name = "Wage classifiction training"
job_description = "First training job"
job_status_message =  "Creating a new job"

job = {
    "name": job_name,
    "description": job_description,
    "status_message": job_status_message,
    "application_id": application_id,
    "job_type": JOB_TYPE_TRAINING,
    "dataset_id": d_id,
    "dataset_version_id": v_id,
    "items": [
      {
        "name": "arch",
        "value": "[200 - 100]",
        "value_type": "text",
        "label": "Architecture"
      },
      {
        "name": "batch_size",
        "value": "32",
        "value_type": "number",
        "label": "Batch Size"
    },
        {
    "name": "nb_epochs",
    "value": "50",
    "value_type": "number",
    "label": "Number of epochs"
}
    ]
}

In [50]:

job = cl.create_job(job)

In [53]:
job = cl.get_job(job["id"]);
print(job["status"])

finished


In [54]:
# Once finished
job

{'id': 'c9de8c8c-9113-4dd3-ab49-dbed8c7af73e',
 'created_at': '2022-09-27T21:18:30.337418+02:00',
 'updated_at': '2022-09-27T21:24:00.168021+02:00',
 'name': 'Wage classifiction training',
 'description': 'First training job',
 'job_type': 'training',
 'application_id': 'd2b25196-dcad-4203-9cbb-87deec0e4f07',
 'status': 'finished',
 'status_message': 'updating training request: finished',
 'user_id': '2b02eb44-8ad0-409c-9515-0405fb25d470',
 'cpu_start_time': '2022-09-27T19:18:32.024237',
 'cpu_end_time': '2022-09-27T19:24:00.164139',
 'gpu_start_time': '',
 'gpu_end_time': '',
 'agent_name': 't1',
 'dataset_id': 'da540de1-48a2-4a86-9c4c-99a73abae6d8',
 'dataset_version_id': '6031678d-a9ed-4fb3-be8c-8a3c23332813',
 'model_id': '580edfb6-c892-4a16-b1f7-89cd6a26dd24',
 'model_version_id': '992ec5d6-b7d4-46db-89fc-22a3ddf23cb9',
 'items': [{'id': 'b25ba689-dc3e-46e4-a380-c57bf21f614a',
   'created_at': '2022-09-27T21:18:30.337756+02:00',
   'updated_at': '2022-09-27T21:24:00.16825+02:00',


In [55]:
# Get created model
model_id = job["model_id"]
model_version_id = job["model_version_id"]

In [56]:
print(model_id)
print(model_version_id)

580edfb6-c892-4a16-b1f7-89cd6a26dd24
992ec5d6-b7d4-46db-89fc-22a3ddf23cb9


In [ ]:
# Predict

In [57]:
model_version = cl.get_model_version(model_id, model_version_id);
model_version

{'id': '992ec5d6-b7d4-46db-89fc-22a3ddf23cb9',
 'created_at': '2022-09-27T21:24:00.042635+02:00',
 'updated_at': '2022-09-27T21:24:00.16219+02:00',
 'name': 'Wage classifiction training',
 'description': '',
 'model_id': '580edfb6-c892-4a16-b1f7-89cd6a26dd24',
 'classes': '',
 'accuracy': 0,
 'user_id': '2b02eb44-8ad0-409c-9515-0405fb25d470',
 'can_inference': True,
 'has_logo': False,
 'logo': '',
 'config': '[{"name": "workclass", "type": "cat"}, {"name": "education", "type": "cat"}, {"name": "marital-status", "type": "cat"}, {"name": "occupation", "type": "cat"}, {"name": "relationship", "type": "cat"}, {"name": "race", "type": "cat"}, {"name": "sex", "type": "cat"}, {"name": "native-country", "type": "cat"}, {"name": "education-num_na", "type": "cat"}, {"name": "age", "type": "cont"}, {"name": "fnlwgt", "type": "cont"}, {"name": "education-num", "type": "cont"}, {"name": "capital-gain", "type": "cont"}, {"name": "capital-loss", "type": "cont"}, {"name": "hours-per-week", "type": "c

In [58]:
import json

config = json.loads(model_version["config"])


In [72]:
for c in config:
    print(c)

{'name': 'workclass', 'type': 'cat'}
{'name': 'education', 'type': 'cat'}
{'name': 'marital-status', 'type': 'cat'}
{'name': 'occupation', 'type': 'cat'}
{'name': 'relationship', 'type': 'cat'}
{'name': 'race', 'type': 'cat'}
{'name': 'sex', 'type': 'cat'}
{'name': 'native-country', 'type': 'cat'}
{'name': 'education-num_na', 'type': 'cat'}
{'name': 'age', 'type': 'cont'}
{'name': 'fnlwgt', 'type': 'cont'}
{'name': 'education-num', 'type': 'cont'}
{'name': 'capital-gain', 'type': 'cont'}
{'name': 'capital-loss', 'type': 'cont'}
{'name': 'hours-per-week', 'type': 'cont'}


In [77]:
inputs = {
    "workclass": "Private", 
    "education": "Assoc-acdm", 
    "marital-status": "Married-civ-spouse", 
    "relationship": 12.0,
    "race": "White",
    "age": 49.0,
    "fnlwgt": 101320.0,
    "education-num": 12.0,
    "occupation": "0",
#     "education-num_na": "0",
    "sex": "Female",
    "native-country": "United-States",
    "fnlwgt": 101320,
    "capital-gain":0,
    "capital-loss": 0,
    "hours-per-week": 40
}


In [78]:
json.dumps(inputs)

'{"workclass": "Private", "education": "Assoc-acdm", "marital-status": "Married-civ-spouse", "relationship": 12.0, "race": "White", "age": 49.0, "fnlwgt": 101320, "education-num": 12.0, "occupation": "0", "sex": "Female", "native-country": "United-States", "capital-gain": 0, "capital-loss": 0, "hours-per-week": 40}'

In [79]:
print(model_id)

580edfb6-c892-4a16-b1f7-89cd6a26dd24


In [80]:
cl.inference(model_id, json.dumps(inputs), input_type="structured")

{'id': '75bb0335-861c-4715-976c-510471d54831',
 'created_at': '2022-09-27T21:36:22.659106277+02:00',
 'updated_at': '2022-09-27T21:36:22.89291321+02:00',
 'name': '{"workclass": "Private", "education": "Assoc-acdm", "marital-status": "Married-civ-spouse", "relationship": 12.0, "race": "White", "age": 49.0, "fnlwgt": 101320, "education-num": 12.0, "occupation": "0", "sex": "Female", "native-country": "United-States", "capital-gain": 0, "capital-loss": 0, "hours-per-week": 40}',
 'description': '',
 'prediction': '<50k',
 'confidence': 0.9017313,
 'model_id': '580edfb6-c892-4a16-b1f7-89cd6a26dd24',
 'model_version_id': '992ec5d6-b7d4-46db-89fc-22a3ddf23cb9',
 'extension': '',
 'user_id': '',
 'error_reported': False,
 'error': '',
 'application_id': 'd2b25196-dcad-4203-9cbb-87deec0e4f07',
 'inference_host': 'structured-pt-1-12-1-fa-2-7-9',
 'inference_time': '228.278543ms',
 'end_to_end_time': '',
 'dataset_item_id': '',
 'result': '',
 'inference_items': None,
 'hidden': False,
 'privac

In [ ]:
# See predictions

In [ ]:
# Add to new version